In [34]:
import pandas as pd
import numpy as np
import joblib
import os
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import KFold
from tqdm import tqdm

pd.options.mode.chained_assignment = None

In [35]:
train_trans = pd.read_csv('data/train_trans.csv')

In [36]:
train_base = pd.read_csv('data/train_base.csv')
test_a_base = pd.read_csv('data/test_a_base.csv')

train_op = pd.read_csv('data/train_op.csv')
test_a_op = pd.read_csv('data/test_a_op.csv')

train_trans = pd.read_csv('data/train_trans.csv')
test_a_trans = pd.read_csv('data/test_a_trans.csv')

train_label = pd.read_csv('data/train_label.csv')

In [100]:
train_base = pd.merge(train_base, train_label, on='user', how='left')

## base处理

In [144]:
train_base_no_nan = train_base.drop(columns=['service3_level'],axis=1)
# 离散值填充众数
train_base_no_nan['sex'].fillna('category 0',inplace=True)
train_base_no_nan['balance_avg'].fillna('level 1',inplace=True)
train_base_no_nan['balance1_avg'].fillna('level 1',inplace=True)
train_base_no_nan['balance2_avg'].fillna('level 1',inplace=True)

In [145]:
user = train_base_no_nan['user'].values
train_base_no_user = train_base_no_nan.drop(columns=['user'],axis=1)

In [168]:
category_columns_name = ['sex','provider','level','verified','agreement1','agreement2','agreement3','agreement4']

pd.get_dummies(train_base_no_user[category_columns_name])

,sex_category 0,sex_category 1,provider_category 0,provider_category 1,provider_category 2,level_category 0,level_category 1,level_category 2,verified_category 0,verified_category 1,agreement1_category 0,agreement1_category 1,agreement2_category 0,agreement2_category 1,agreement3_category 0,agreement3_category 1,agreement4_category 0,agreement4_category 1
0,0,1,1,0,0,0,0,1,1,0,1,0,1,0,0,1,1,0
1,0,1,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1
2,1,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,1,0
3,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0
4,1,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
47777,1,0,1,0,0,0,1,0,1,0,1,0,1,0,1,0,1,0
47778,1,0,0,1,0,0,1,0,1,0,1,0,1,0,1,0,1,0
47779,1,0,1,0,0,0,0,1,1,0,1,0,1,0,1,0,1,0
47780,1,0,1,0,0,0,0,1,1,0,1,0,0,1,1,0,1,0


In [146]:
# 少的做one-hot
category_columns_name = ['sex','provider','level','verified','regist_type','agreement1','agreement2','agreement3','agreement4']
# ,'city','province'
ohe = OneHotEncoder()
ohe_encoder = ohe.fit(train_base_no_user[category_columns_name])
one_hot_value = ohe_encoder.transform(train_base_no_user[category_columns_name]).toarray()

one_hot_df = pd.DataFrame(one_hot_value)
joblib.dump(ohe_encoder,'ohe_encoder.pkl')

['ohe_encoder.pkl']

In [147]:
train_base_categoty = train_base_no_user.select_dtypes('O')

for c in list(train_base_categoty.columns[0:9])+['service3']:
    train_base_categoty[c]=train_base_categoty[c].apply(lambda x:int(x.strip().split(' ')[1]))
    
for c in list(train_base_categoty.columns[11:17])+list(train_base_categoty.columns[18:25]):
    train_base_categoty[c]=train_base_categoty[c].apply(lambda x:int(x.strip().split(' ')[1]))

In [27]:
def mean_coding(data, feature, target='label'):
    n_folds = 10
    n_inner_folds = 5
    mean_coded = pd.Series()

    # 所有数据的label均值
    default_mean = data[target].mean()
    kf = KFold(n_splits=n_folds, shuffle=True)

    out_mean_cv = pd.DataFrame()
    split = 0
    # 对所有数据做CV
    for in_fold, out_fold in tqdm(kf.split(data[feature])):

        impact_coded_cv = pd.Series()
        kf_inner = KFold(n_splits=n_inner_folds, shuffle=True)
        inner_split = 0
        inner_mean_cv = pd.DataFrame()

        # in_fold数据的label均值
        default_inner_mean = data.iloc[in_fold][target].mean()

        # 对in_fold数据做CV
        for in_fold_inner, out_fold_inner in kf_inner.split(data.iloc[in_fold]):
            # The mean to apply to the inner oof split (a 1/n_folds % based on the rest)

            # 对in_fold_inner做group_by求出mean:feature-mean
            in_fold_inner_mean = data.iloc[in_fold_inner].groupby(by=feature)[target].mean()

            # in_fold的mean使用cv后的in_fold_inner的mean值，如果没有在里面，就用in_fold的label均值
            impact_coded_cv = impact_coded_cv.append(data.iloc[in_fold].apply(
                lambda x: in_fold_inner_mean[x[feature]]
                if x[feature] in in_fold_inner_mean.index
                else default_inner_mean
                , axis=1))

            # Also populate mapping (this has all group -> mean for all inner CV folds)
            inner_mean_cv = inner_mean_cv.join(pd.DataFrame(in_fold_inner_mean), rsuffix=inner_split, how='outer')
            inner_mean_cv.fillna(value=default_inner_mean, inplace=True)
            inner_split += 1

        # Also populate mapping
        out_mean_cv = out_mean_cv.join(pd.DataFrame(inner_mean_cv), rsuffix=split, how='outer')
        out_mean_cv.fillna(value=default_mean, inplace=True)
        split += 1

        mean_coded = mean_coded.append(data.iloc[out_fold].apply(
            lambda x: inner_mean_cv.loc[x[feature]].mean()
            if x[feature] in inner_mean_cv.index
            else default_mean
            , axis=1))

    return mean_coded, out_mean_cv.mean(axis=1), default_mean

In [31]:
city_mean, city_mapping, city_default_means = mean_coding(train_base_no_user, 'city')


0it [00:00, ?it/s]
1it [00:05,  5.93s/it]
2it [00:11,  5.94s/it]
3it [00:17,  5.94s/it]
4it [00:23,  5.95s/it]
5it [00:29,  5.94s/it]
6it [00:35,  5.93s/it]
7it [00:41,  5.95s/it]
8it [00:47,  5.97s/it]
9it [00:53,  5.96s/it]
10it [00:59,  5.96s/it]


In [32]:
province_mean, province_mapping, province_default_means = mean_coding(train_base_no_user, 'province')


0it [00:00, ?it/s]
1it [00:06,  6.10s/it]
2it [00:11,  6.04s/it]
3it [00:17,  5.98s/it]
4it [00:23,  5.94s/it]
5it [00:29,  5.91s/it]
6it [00:35,  5.88s/it]
7it [00:41,  5.87s/it]
8it [00:47,  5.86s/it]
9it [00:52,  5.87s/it]
10it [00:58,  5.88s/it]


In [ ]:
regist_type_mean, regist_type_mapping, regist_type_default_means = mean_coding(train_base_no_user, 'regist_type')

In [33]:
joblib.dump(city_mean,'city_mean')
joblib.dump(province_mean,'province_mean')

['province_mean']

In [148]:
train_base_categoty['city'] = city_mean
train_base_categoty['province'] = province_mean

In [9]:
# cumsu = train_base_no_user.groupby('city')['label'].cumsum() - train_base_no_user['label']
# cumcnt = train_base_no_user.groupby('city').cumcount()+1e-10
# train_base_categoty['city'] = cumsu/cumcnt

# cumsu = train_base_no_user.groupby('province')['label'].cumsum() - train_base_no_user['label']
# cumcnt = train_base_no_user.groupby('province').cumcount()+1e-10
# train_base_categoty['province'] = cumsu/cumcnt

In [ ]:
# city_dic = dict(train_base_no_user.groupby('city')['label'].sum()/train_base_no_user.groupby('city')['label'].count())
# province_dic = dict(train_base_no_user.groupby('province')['label'].sum()/train_base_no_user.groupby('province')['label'].count())

# train_base_categoty['city'] = train_base_categoty['city'].apply(lambda x:city_dic[x])
# train_base_categoty['province'] = train_base_categoty['province'].apply(lambda x:province_dic[x])

In [ ]:
# joblib.dump(city_dic,'city_dic.pkl')
# joblib.dump(province_dic,'province_dic.pkl')

In [149]:
# train_base_categoty.drop(columns=['province','city'],axis=1,inplace=True)
train_base_categoty.drop(columns=category_columns_name,axis=1,inplace=True)

In [ ]:
train_base_categoty

In [ ]:
# joblib.dump(ohe_encoder,'one_encoder.pkl')

In [150]:
number_value = train_base_no_user.select_dtypes('int64').values
df_value= pd.DataFrame(number_value)
df_value.columns = train_base_no_user.select_dtypes('int64').columns

In [151]:
df_value['product7_success_cnt'] = df_value['product7_cnt']-df_value['product7_fail_cnt']
df_value['card_cnt'] = df_value['card_a_cnt']+df_value['card_b_cnt']+df_value['card_c_cnt']+df_value['card_d_cnt']
df_value['ip_cnt_avg'] = df_value['ip_cnt']/df_value['login_days_cnt']
df_value['login_cnt_period']=df_value['login_cnt_period1']+df_value['login_cnt_period2']
df_value['login_cnt_period1_avg']=df_value['login_cnt_period1']/df_value['login_days_cnt']
df_value['login_cnt_period2_avg']=df_value['login_cnt_period2']/df_value['login_days_cnt']
df_value['login_cnt_period_avg']=df_value['login_cnt_period']/df_value['login_days_cnt']
df_value['using_time_avg'] = df_value['using_time']/df_value['login_days_cnt']
df_value['product7_success_cnt_avg'] = df_value['product7_success_cnt']/df_value['login_days_cnt']
df_value['product7_cnt_avg']=df_value['product7_cnt']/df_value['login_days_cnt']
df_value['product7_fail_cnt_avg']=df_value['product7_fail_cnt']/df_value['login_days_cnt']
df_value['service_cnt']=df_value['service1_cnt']+df_value['service2_cnt']
df_value['service_cnt_avg']= df_value['service_cnt']/df_value['login_days_cnt']
df_value['service_amt_cnt_avg']=df_value['service1_amt']/df_value['service_cnt']

In [152]:
# 归一化
df_value = (df_value-df_value.min())/(df_value.max()-df_value.min())
# df_value = (df_value-df_value.mean())/df_value.std()

In [153]:
# df_one_hot = pd.DataFrame(one_hot_value)

df_base = pd.DataFrame()
df_base['user'] = user

df_base = pd.concat([df_base, train_base_categoty,df_value,one_hot_df],axis=1)

In [154]:
df_base.shape

(47782, 78)

In [155]:
df_base.head()

,user,province,city,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,service3,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount,age,using_time,card_a_cnt,card_b_cnt,card_c_cnt,op1_cnt,op2_cnt,card_d_cnt,agreement_total,service1_cnt,service1_amt,service2_cnt,acc_count,login_cnt_period1,login_cnt_period2,ip_cnt,login_cnt_avg,login_days_cnt,product7_cnt,product7_fail_cnt,label_x,label_y,product7_success_cnt,card_cnt,ip_cnt_avg,login_cnt_period,login_cnt_period1_avg,login_cnt_period2_avg,login_cnt_period_avg,using_time_avg,product7_success_cnt_avg,product7_cnt_avg,product7_fail_cnt_avg,service_cnt,service_cnt_avg,service_amt_cnt_avg,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25
0,Train_06800,0.230346,0.205046,1,1,1,1,1,1,0,1,1,1,0,0,1,0.235862,0.415094,0.057692,0.098361,0.000000,0.091346,0.027397,0.0,0.203252,0.0,0.0,0.0,0.003762,0.004402,0.003142,0.010150,0.021058,0.078467,0.00000,0.0,0.0,0.0,0.000000,0.057692,0.225496,0.003810,0.004135,0.002801,0.003509,0.883054,0.000000,0.834355,0.834355,0.0,0.899806,0.000008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0
1,Train_23487,0.197903,0.208129,19,19,1,1,20,5,0,1,1,1,0,0,1,0.260690,0.245283,0.125000,0.213115,0.000000,0.062500,0.027397,0.0,0.252033,0.0,0.0,0.0,0.001736,0.004759,0.004061,0.045406,0.012243,0.167883,0.04878,0.0,1.0,1.0,0.857143,0.125000,0.235239,0.004431,0.004076,0.003265,0.003695,0.783694,0.853963,0.762022,0.751909,0.0,0.810893,0.000008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0
2,Train_36880,0.231362,0.257062,7,5,14,11,1,5,0,1,1,1,0,0,1,0.202759,0.320755,0.125000,0.213115,0.000000,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.005498,0.019068,0.010166,0.123932,0.018119,0.322993,0.04878,0.0,1.0,1.0,0.857143,0.125000,0.270565,0.014887,0.017896,0.008655,0.013557,0.645769,0.851046,0.619739,0.609661,0.0,0.657486,0.000008,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
3,Train_35392,0.201042,0.185761,11,7,1,1,12,4,0,1,1,1,0,0,1,0.320000,0.018868,0.057692,0.098361,0.000000,0.000000,0.000000,0.0,0.203252,0.0,0.0,0.0,0.005498,0.003347,0.003779,0.049145,0.012243,0.155109,0.04878,0.0,0.0,0.0,0.857143,0.057692,0.243241,0.003550,0.002696,0.003046,0.002860,0.775081,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0
4,Train_35057,0.316504,0.263127,8,9,1,1,9,5,0,2,8,1,0,0,1,0.344828,0.415094,0.125000,0.098361,0.070588,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.001736,0.003646,0.001435,0.032585,0.002938,0.155109,0.04878,0.0,1.0,1.0,0.857143,0.120192,0.225519,0.002607,0.003001,0.000651,0.001898,0.811092,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0


## OP处理

In [49]:
train_op.head()

,user,op_type,op_mode,op_device,ip,net_type,channel,ip_3,tm_diff
0,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:22.000000000
1,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:21.000000000
2,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:23.000000000
3,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:26.000000000
4,Train_36517,b26bc49195bd79cf,87ee0bdf333a54da,92dc8b3f9a9ea13b,bbb0af60b941370b,116a2503b987ea81,4e1ff124e1e6adc8,1e46c177cd9d539a,11 days 09:38:41.000000000


In [50]:
op_df = pd.DataFrame()
group = train_op.groupby(['user']).count()
op_df['user'] = group.index

In [51]:
count_column=train_op.columns[1:-1]
for column in count_column:
    print(column)
#     des_df = train_op.groupby('user')[column].describe()
#     op_df['op_'+column+'_count'] = des_df['count'].values
#     op_df['op_'+column+'_n'] = des_df['unique'].values
#     op_df['op_'+column+'_fre'] = des_df['freq'].values
    
    op_df['op_'+column+'_count'] = train_op.groupby('user')[column].count().values
    op_df['op_'+column+'_count'] = train_op.groupby('user')[column].nunique().values
    

op_type
op_mode
op_device
ip
net_type
channel
ip_3


In [52]:
from datetime import timedelta
def timedelta2sec(delta):
    day = delta.split('days')[0].strip()
    h,m,s = delta.split('days')[1].strip().split(':')
    sec=timedelta(days=int(day),hours=int(h),minutes=int(m),seconds=float(s)).total_seconds()
    return sec

train_op['time_diff_sec'] = train_op['tm_diff'].apply(timedelta2sec)

for operate in ['max','min','mean','median','std']:
    op_df['op_time_'+operate]=train_op.groupby('user')['time_diff_sec'].agg(operate).values

In [53]:
op_df.fillna(0,inplace=True)

In [54]:
op_df.head()

,user,op_op_type_count,op_op_mode_count,op_op_device_count,op_ip_count,op_net_type_count,op_channel_count,op_ip_3_count,op_time_max,op_time_min,op_time_mean,op_time_median,op_time_std
0,Train_00000,11,12,2,3,3,4,3,683286.0,219874.0,3.058014e+05,263361.0,125051.535485
1,Train_00001,3,3,1,2,2,3,2,1263768.0,409809.0,6.598048e+05,438553.5,385614.886428
2,Train_00002,3,3,1,1,2,3,1,1283043.0,1282741.0,1.282845e+06,1282751.5,139.053690
3,Train_00003,8,8,2,18,2,3,5,1278388.0,199014.0,9.408690e+05,957412.0,247893.797669
4,Train_00004,2,2,1,1,1,2,1,631021.0,630973.0,6.309866e+05,630975.0,20.659138


In [55]:
op_df.shape

(41892, 13)

## 交易处理

In [56]:
train_trans.head()

,user,platform,tunnel_in,tunnel_out,amount,type1,ip,type2,ip_3,tm_diff
0,Train_13770,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,185784,45a1168437c708ff,NaN,11a213398ee0c623,NaN,19 days 09:02:45.000000000
1,Train_13770,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,391769,45a1168437c708ff,NaN,11a213398ee0c623,NaN,19 days 09:03:58.000000000
2,Train_08351,46c69cbbce5f1568,b2e7fa260df4998d,6ee790756007e69a,27740,674e8d5860bc033d,f10a09fe9e522a47,11a213398ee0c623,ee386d6f9fe45d0d,18 days 11:06:49.000000000
3,Train_08351,42573d7287a8c9c2,NaN,6ee790756007e69a,36689,f67d4b5a05a1352a,NaN,NaN,NaN,26 days 09:52:51.000000000
4,Train_08351,42573d7287a8c9c2,NaN,6ee790756007e69a,30746,f67d4b5a05a1352a,NaN,NaN,NaN,26 days 07:50:05.000000000


In [57]:
trans_df = pd.DataFrame()
group = train_trans.groupby(['user']).count()
trans_df['user'] = group.index

### 离散列，全部统计有几个类别

In [58]:
count_column = list(train_trans.columns[1:4])+list(train_trans.columns[5:9]) 
for column in count_column:
    print(column)
    trans_df['trans_'+column+'_count'] = train_trans.groupby('user')[column].count().values
    trans_df['trans_'+column+'_count'] = train_trans.groupby('user')[column].nunique().values
#     des_df = train_trans.groupby('user')[column].describe()
#     trans_df['trans_'+column+'_count'] = des_df['count'].values
#     trans_df['trans_'+column+'_n'] = des_df['unique'].values
#     trans_df['trans_'+column+'_fre'] = des_df['freq'].values

platform
tunnel_in
tunnel_out
type1
ip
type2
ip_3


### 处理amount

In [59]:
for operate in ['max','min','mean','median','std']:
    trans_df['trans_amount_'+operate]=train_trans.groupby(['user'])['amount'].agg(operate).values

### 处理时间

In [60]:
train_trans['time_diff_sec'] = train_trans['tm_diff'].apply(timedelta2sec)

for operate in ['max','min','mean','median','std']:
    trans_df['trans_time_'+operate]=train_trans.groupby('user')['time_diff_sec'].agg(operate).values

### 标准差、freq有空值，用0填充

In [61]:
trans_df.fillna(0,inplace=True)

In [62]:
trans_df.head()

,user,trans_platform_count,trans_tunnel_in_count,trans_tunnel_out_count,trans_type1_count,trans_ip_count,trans_type2_count,trans_ip_3_count,trans_amount_max,trans_amount_min,trans_amount_mean,trans_amount_median,trans_amount_std,trans_time_max,trans_time_min,trans_time_mean,trans_time_median,trans_time_std
0,Train_00000,2,1,1,4,2,1,2,267542,24798,53330.307692,30746.0,65274.138488,2478067.0,1169773.0,1.553383e+06,1627246.0,3.970320e+05
1,Train_00001,2,1,1,2,1,1,1,36098,36098,36098.000000,36098.0,0.000000,1821027.0,289554.0,1.055290e+06,1055290.5,1.082915e+06
2,Train_00002,2,1,1,3,3,1,3,162423,24980,57329.583333,34541.5,49227.699521,2665430.0,575019.0,1.454428e+06,625491.0,1.069315e+06
3,Train_00003,2,1,1,5,2,1,2,188880,25315,61652.454545,36689.0,62929.509668,2563665.0,830115.0,2.243363e+06,2562619.0,6.976849e+05
4,Train_00004,1,0,1,1,0,0,0,36689,36689,36689.000000,36689.0,0.000000,2014022.0,2014022.0,2.014022e+06,2014022.0,0.000000e+00


In [63]:
trans_df.shape

(41560, 18)

## 数据合并

In [156]:
# train_df = pd.merge(df_base, train_label, on='user', how='left')
train_df = pd.merge(df_base, op_df, on='user', how='left')
train_df = pd.merge(train_df, trans_df, on='user', how='left')


In [157]:
train_df.shape

(47782, 107)

In [158]:
'label' in train.columns

False

In [159]:
train=train_df

In [160]:
train.head()

,user,province,city,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,service3,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount,age,using_time,card_a_cnt,card_b_cnt,card_c_cnt,op1_cnt,op2_cnt,card_d_cnt,agreement_total,service1_cnt,service1_amt,service2_cnt,acc_count,login_cnt_period1,login_cnt_period2,ip_cnt,login_cnt_avg,login_days_cnt,product7_cnt,product7_fail_cnt,label_x,label_y,product7_success_cnt,card_cnt,ip_cnt_avg,login_cnt_period,login_cnt_period1_avg,login_cnt_period2_avg,login_cnt_period_avg,using_time_avg,product7_success_cnt_avg,product7_cnt_avg,product7_fail_cnt_avg,service_cnt,service_cnt_avg,service_amt_cnt_avg,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,op_op_type_count,op_op_mode_count,op_op_device_count,op_ip_count,op_net_type_count,op_channel_count,op_ip_3_count,op_time_max,op_time_min,op_time_mean,op_time_median,op_time_std,trans_platform_count,trans_tunnel_in_count,trans_tunnel_out_count,trans_type1_count,trans_ip_count,trans_type2_count,trans_ip_3_count,trans_amount_max,trans_amount_min,trans_amount_mean,trans_amount_median,trans_amount_std,trans_time_max,trans_time_min,trans_time_mean,trans_time_median,trans_time_std
0,Train_06800,0.230346,0.205046,1,1,1,1,1,1,0,1,1,1,0,0,1,0.235862,0.415094,0.057692,0.098361,0.000000,0.091346,0.027397,0.0,0.203252,0.0,0.0,0.0,0.003762,0.004402,0.003142,0.010150,0.021058,0.078467,0.00000,0.0,0.0,0.0,0.000000,0.057692,0.225496,0.003810,0.004135,0.002801,0.003509,0.883054,0.000000,0.834355,0.834355,0.0,0.899806,0.000008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,15.0,17.0,1.0,1.0,1.0,3.0,1.0,1238195.0,983947.0,1.065893e+06,1077918.5,89914.677544,1.0,1.0,1.0,1.0,0.0,1.0,0.0,33728.0,30746.0,31551.250000,30865.5,1455.533665,2620604.0,2367781.0,2.494192e+06,2494192.5,145967.427107
1,Train_23487,0.197903,0.208129,19,19,1,1,20,5,0,1,1,1,0,0,1,0.260690,0.245283,0.125000,0.213115,0.000000,0.062500,0.027397,0.0,0.252033,0.0,0.0,0.0,0.001736,0.004759,0.004061,0.045406,0.012243,0.167883,0.04878,0.0,1.0,1.0,0.857143,0.125000,0.235239,0.004431,0.004076,0.003265,0.003695,0.783694,0.853963,0.762022,0.751909,0.0,0.810893,0.000008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1025699.0,1025674.0,1.025683e+06,1025677.0,13.650397,1.0,1.0,1.0,2.0,1.0,1.0,1.0,503457.0,380126.0,441791.500000,441791.5,87208.186431,980425.0,980181.0,9.803030e+05,980303.0,172.534055
2,Train_36880,0.231362,0.257062,7,5,14,11,1,5,0,1,1,1,0,0,1,0.202759,0.320755,0.125000,0.213115,0.000000,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.005498,0.019068,0.010166,0.123932,0.018119,0.322993,0.04878,0.0,1.0,1.0,0.857143,0.125000,0.270565,0.014887,0.017896,0.008655,0.013557,0.645769,0.851046,0.619739,0.609661,0.0,0.657486,0.000008,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,4.0,1.0,1.0,2.0,3.0,1.0,579758.0,579581.0,5.796529e+05,579643.0,61.180017,3.0,1.0,1.0,4.0,2.0,2.0,2.0,787672.0,24798.0,307374.111111,166658.0,288304.779934,1962149.0,123248.0,1.612149e+06,1962074.0,706810.862319
3,Train_35392,0.201042,0.185761,11,7,1,1,12,4,0,1,1,1,0,0,1,0.320000,0.018868,0.057692,0.098361,0.000000,0.000000,0.000000,0.0,0.203252,0.0,0.0,0.0,0.005498,0.003347,0.003779,0.049145,0.012243,0.155109,0.04878,0.0,0.0,0.0,0.857143,0.057692,0.243241,0.003550,0.002696,0.003046,0.002860,0.775081,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,10.0,11.0,1.0,10.0,3.0,3.0,7.0,907905.0,61355.0,4.752282e+05,545643.0,310441.845118,2.0,1.0,1.0,3.0,5.0,1.0,4.0,39630.0,24743.0,27326.555556,25923.0,4026.466663,2290284.0,1443970.0,1.875648e+06,1928058.0,291556.617186
4,Train_35057,0.316504,0.263127,8,9,1,1,9,5,0,2,8,1,0,0,1,0.344828,0.415094,0.125000,0.098361,0.0705

In [124]:
# def label_smoothing(inputs, epsilon=0.1):
#     K = inputs.shape[-1]
#     return ((1-epsilon) * inputs) + (epsilon / K)

In [161]:
length = len(train)
train_length = int(0.90*length)

x = train.drop(columns=['user','label_x','label_y'],axis=1)[0:train_length]
y = train['label_x'].values[0:train_length]
# y = label_smoothing(y)
# 
valid_x = train.drop(columns=['user','label_x','label_y'],axis=1)[train_length:]
valid_y = train['label_x'].values[train_length:]
# valid_y = label_smoothing(valid_y)

In [162]:
x.shape

(43003, 104)

In [163]:
x = x.fillna(0)
valid_x = valid_x.fillna(0)

In [164]:
x.head()

,province,city,balance,balance_avg,balance1,balance1_avg,balance2,balance2_avg,service3,product1_amount,product2_amount,product3_amount,product4_amount,product5_amount,product6_amount,age,using_time,card_a_cnt,card_b_cnt,card_c_cnt,op1_cnt,op2_cnt,card_d_cnt,agreement_total,service1_cnt,service1_amt,service2_cnt,acc_count,login_cnt_period1,login_cnt_period2,ip_cnt,login_cnt_avg,login_days_cnt,product7_cnt,product7_fail_cnt,product7_success_cnt,card_cnt,ip_cnt_avg,login_cnt_period,login_cnt_period1_avg,login_cnt_period2_avg,login_cnt_period_avg,using_time_avg,product7_success_cnt_avg,product7_cnt_avg,product7_fail_cnt_avg,service_cnt,service_cnt_avg,service_amt_cnt_avg,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,op_op_type_count,op_op_mode_count,op_op_device_count,op_ip_count,op_net_type_count,op_channel_count,op_ip_3_count,op_time_max,op_time_min,op_time_mean,op_time_median,op_time_std,trans_platform_count,trans_tunnel_in_count,trans_tunnel_out_count,trans_type1_count,trans_ip_count,trans_type2_count,trans_ip_3_count,trans_amount_max,trans_amount_min,trans_amount_mean,trans_amount_median,trans_amount_std,trans_time_max,trans_time_min,trans_time_mean,trans_time_median,trans_time_std
0,0.230346,0.205046,1,1,1,1,1,1,0,1,1,1,0,0,1,0.235862,0.415094,0.057692,0.098361,0.000000,0.091346,0.027397,0.0,0.203252,0.0,0.0,0.0,0.003762,0.004402,0.003142,0.010150,0.021058,0.078467,0.00000,0.0,0.000000,0.057692,0.225496,0.003810,0.004135,0.002801,0.003509,0.883054,0.000000,0.834355,0.834355,0.0,0.899806,0.000008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,0.0,15.0,17.0,1.0,1.0,1.0,3.0,1.0,1238195.0,983947.0,1.065893e+06,1077918.5,89914.677544,1.0,1.0,1.0,1.0,0.0,1.0,0.0,33728.0,30746.0,31551.250000,30865.5,1455.533665,2620604.0,2367781.0,2.494192e+06,2494192.5,145967.427107
1,0.197903,0.208129,19,19,1,1,20,5,0,1,1,1,0,0,1,0.260690,0.245283,0.125000,0.213115,0.000000,0.062500,0.027397,0.0,0.252033,0.0,0.0,0.0,0.001736,0.004759,0.004061,0.045406,0.012243,0.167883,0.04878,0.0,0.857143,0.125000,0.235239,0.004431,0.004076,0.003265,0.003695,0.783694,0.853963,0.762022,0.751909,0.0,0.810893,0.000008,0.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,0.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1.0,1025699.0,1025674.0,1.025683e+06,1025677.0,13.650397,1.0,1.0,1.0,2.0,1.0,1.0,1.0,503457.0,380126.0,441791.500000,441791.5,87208.186431,980425.0,980181.0,9.803030e+05,980303.0,172.534055
2,0.231362,0.257062,7,5,14,11,1,5,0,1,1,1,0,0,1,0.202759,0.320755,0.125000,0.213115,0.000000,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.005498,0.019068,0.010166,0.123932,0.018119,0.322993,0.04878,0.0,0.857143,0.125000,0.270565,0.014887,0.017896,0.008655,0.013557,0.645769,0.851046,0.619739,0.609661,0.0,0.657486,0.000008,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,4.0,4.0,1.0,1.0,2.0,3.0,1.0,579758.0,579581.0,5.796529e+05,579643.0,61.180017,3.0,1.0,1.0,4.0,2.0,2.0,2.0,787672.0,24798.0,307374.111111,166658.0,288304.779934,1962149.0,123248.0,1.612149e+06,1962074.0,706810.862319
3,0.201042,0.185761,11,7,1,1,12,4,0,1,1,1,0,0,1,0.320000,0.018868,0.057692,0.098361,0.000000,0.000000,0.000000,0.0,0.203252,0.0,0.0,0.0,0.005498,0.003347,0.003779,0.049145,0.012243,0.155109,0.04878,0.0,0.857143,0.057692,0.243241,0.003550,0.002696,0.003046,0.002860,0.775081,0.854204,0.773783,0.763667,0.0,0.823573,0.000008,1.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,1.0,0.0,10.0,11.0,1.0,10.0,3.0,3.0,7.0,907905.0,61355.0,4.752282e+05,545643.0,310441.845118,2.0,1.0,1.0,3.0,5.0,1.0,4.0,39630.0,24743.0,27326.555556,25923.0,4026.466663,2290284.0,1443970.0,1.875648e+06,1928058.0,291556.617186
4,0.316504,0.263127,8,9,1,1,9,5,0,2,8,1,0,0,1,0.344828,0.415094,0.125000,0.098361,0.070588,0.028846,0.027397,0.0,0.349593,0.0,0.0,0.0,0.001736,0.003646,0.001435,0.032585,0.002938,0.155109,0.04878,0.0,0

In [165]:
from lightgbm import LGBMClassifier
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression

# model = LogisticRegression()
# model.fit(x,y)

model = LGBMClassifier(n_estimators=1000, 
                       learning_rate=0.04,
                       subsample=0.8,
                       colsample_bytree=0.8,
                       reg_alpha=100,
                       reg_lambda=100)
model.fit(x, y,
          eval_set=(valid_x, valid_y),
          early_stopping_rounds=5
          )

prediction = model.predict_proba(valid_x)[:,1]
auc = roc_auc_score(valid_y,prediction)
auc

[1]	valid_0's binary_logloss: 0.562606
Training until validation scores don't improve for 5 rounds
[2]	valid_0's binary_logloss: 0.56133
[3]	valid_0's binary_logloss: 0.560121
[4]	valid_0's binary_logloss: 0.55905
[5]	valid_0's binary_logloss: 0.558052
[6]	valid_0's binary_logloss: 0.55709
[7]	valid_0's binary_logloss: 0.55623
[8]	valid_0's binary_logloss: 0.555421
[9]	valid_0's binary_logloss: 0.554567
[10]	valid_0's binary_logloss: 0.553838
[11]	valid_0's binary_logloss: 0.553123
[12]	valid_0's binary_logloss: 0.552352
[13]	valid_0's binary_logloss: 0.55169
[14]	valid_0's binary_logloss: 0.551133
[15]	valid_0's binary_logloss: 0.550483
[16]	valid_0's binary_logloss: 0.549814
[17]	valid_0's binary_logloss: 0.549268
[18]	valid_0's binary_logloss: 0.548673
[19]	valid_0's binary_logloss: 0.548218
[20]	valid_0's binary_logloss: 0.547701
[21]	valid_0's binary_logloss: 0.547179
[22]	valid_0's binary_logloss: 0.546941
[23]	valid_0's binary_logloss: 0.5466
[24]	valid_0's binary_logloss: 0.546

[230]	valid_0's binary_logloss: 0.53086
[231]	valid_0's binary_logloss: 0.530856
[232]	valid_0's binary_logloss: 0.530839
Early stopping, best iteration is:
[227]	valid_0's binary_logloss: 0.530802


0.6739886276576594

In [93]:
joblib.dump(model,'model.pkl')

['model.pkl']

In [166]:
importance_df = pd.DataFrame({
        'column': x.columns,
        'importance': model.feature_importances_,
    }).sort_values(by='importance')[-100:]

In [167]:
importance_df

,column,importance
46,service_cnt,0
48,service_amt_cnt_avg,0
67,18,0
13,product5_amount,0
25,service1_amt,0
52,3,0
12,product4_amount,0
54,5,0
57,8,0
58,9,0


In [ ]:
# select_feature=importance_df['column']

# x = x[select_feature]
# valid_x = valid_x[select_feature]

In [ ]:
# from catboost import CatBoostClassifier


# model = CatBoostClassifier(
#     iterations=500,
#     random_seed=42,
#     logging_level='Silent'
# )

# model.fit(
#     x.values, y,
#     eval_set=(valid_x.values, valid_y),
# #     logging_level='Verbose',  # you can uncomment this for text output
#     plot=True
# )

# prediction = model.predict_proba(valid_x)[:,1]
# auc = roc_auc_score(valid_y,prediction)
# auc

baseline:
0.6769352059269422

添加trans amount的扩展变量：
0.6782946345739088

添加trans ip3的扩展变量：
0.6791192236191647

添加trans type直接数值化：
0.6796788342378327

添加所有的des特征，并修改了学习率：
0.6823851869318595

添加avg特征：
0.6831416556804824

In [75]:
#显示所有列
pd.set_option('display.max_columns', None)
#显示所有行
pd.set_option('display.max_rows', 100)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',100)
